In [6]:
import pandas as pd
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.preprocessing import StandardScaler,LabelEncoder,OneHotEncoder
from sklearn.pipeline import Pipeline

from scikeras.wrappers import KerasClassifier
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping
import pickle

In [11]:
 data=pd.read_csv('Churn_Modelling.csv')
data = data.drop(['RowNumber','CustomerId','Surname'],axis=1)

label_encoder_gender=LabelEncoder()
data['Gender']=label_encoder_gender.fit_transform(data['Gender'])


onehot_encoder_geo=OneHotEncoder(handle_unknown='ignore')
geo_encoded=onehot_encoder_geo.fit_transform(data[['Geography']]).toarray()
geo_encoded_df=pd.DataFrame(geo_encoded,columns=onehot_encoder_geo.get_feature_names_out(['Geography']))

data=pd.concat([data.drop('Geography',axis=1),geo_encoded_df],axis=1)
x=data.drop('Exited',axis=1)
y=data['Exited']

x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=42)

scaler=StandardScaler()
x_train=scaler.fit_transform(x_train)
x_test=scaler.transform(x_test)

with open('label_encoder_gender.pkl', 'wb') as file:
    pickle.dump(label_encoder_gender, file)
with open('onehot_encoder_geo.pkl', 'wb') as file:
    pickle.dump(onehot_encoder_geo, file)
with open('scaler.pkl', 'wb') as file:
    pickle.dump(scaler, file)




In [13]:
# define a function to create the model and try different parameter(KerasClassifier)
 # Define a function to create the model with configurable parameters
def create_model(neurons=32, layer=1):
    model = Sequential()
    model.add(Dense(neurons, activation='relu', input_shape=(x_train.shape[1],)))
    
    for _ in range(layer - 1):
        model.add(Dense(neurons, activation='relu'))
    
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer='adam', loss="binary_crossentropy", metrics=['accuracy'])
    return model


In [14]:
## create a keras classifier
model=KerasClassifier(build_fn=create_model,epochs=50,batch_size=10,verbose=0)

In [15]:
# define the grid search parameters
param_grid={
    'neurons':[16,32,64,128],
    'layers':[1,2],
    'epochs':[50,100]
}

In [18]:
param_grid = {
    "model__neurons": [16, 32, 64],
    "model__layer": [1, 2, 3],
    "batch_size": [32, 64],
    "epochs": [10, 20]
}

grid = GridSearchCV(estimator=model, param_grid=param_grid, cv=3, n_jobs=-1, verbose=1)
grid_result = grid.fit(x_train, y_train)

print(f"Best accuracy: {grid_result.best_score_}")
print(f"Best parameters: {grid_result.best_params_}")


Fitting 3 folds for each of 36 candidates, totalling 108 fits


c:\Users\KIIT\Desktop\ann classification\venv\lib\site-packages\scikeras\wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


Best accuracy: 0.8581243079798822
Best parameters: {'batch_size': 64, 'epochs': 10, 'model__layer': 3, 'model__neurons': 64}
